<a href="https://colab.research.google.com/github/muajnstu/Multi-Class-Classification-of-YouTube-Videos-Using-A-BERT-enhanced-Machine-Learning-approach/blob/main/Embedding%2BLSTM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
df = pd.read_csv('https://raw.githubusercontent.com/muajnstu/ML-Datasets/refs/heads/main/Youtube%20Video%20Dataset.csv')

# Robust category mapping
category_map = {
    "travel blog": 0,
    "science&technology": 1,
    "food": 2,
    "art&music": 3,
    "manufacturing": 4,
    "history": 5
}
df["Category"] = df["Category"].str.lower().str.strip().map(category_map)

# Shuffle and process
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df['processed_text'] = df['Title'].astype(str) + " " + df['Description'].astype(str)

def extract_txt(text):
    match = re.search(r"(?<=\s\-\s).*", str(text))
    return match.group(0) if match else text

df['processed_text'] = df['processed_text'].apply(extract_txt)

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['processed_text'] = df['processed_text'].apply(clean_text)
df.rename(columns={'processed_text': 'text'}, inplace=True)

# Split the data
X_train, X_temp, y_train, y_temp = train_test_split(df['text'], df['Category'], test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Tokenization & Sequence Padding
max_words = 10000
max_len = 128

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

# Model: Embedding + LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    tf.keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.build(input_shape=(None, max_len))
model.summary()


model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training
history = model.fit(
    X_train_pad, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_pad, y_val),
    callbacks=[early_stopping]
)

# Evaluation
test_loss, test_acc = model.evaluate(X_test_pad, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

y_pred_proba = model.predict(X_test_pad)
y_pred = np.argmax(y_pred_proba, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=list(category_map.keys())))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 128, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,420,230 (5.42 MB)

 Trainable params: 1,420,230 (5.42 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 120s 413ms/step - accuracy: 0.1911 - loss: 1.7891 - val_accuracy: 0.2721 - val_loss: 1.6860
Epoch 2/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 138s 399ms/step - accuracy: 0.2832 - loss: 1.6997 - val_accuracy: 0.2988 - val_loss: 1.6670
Epoch 3/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 142s 399ms/step - accuracy: 0.2907 - loss: 1.6539 - val_accuracy: 0.2765 - val_loss: 1.7075
Epoch 4/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 144s 406ms/step - accuracy: 0.2967 - loss: 1.6440 - val_accuracy: 0.3434 - val_loss: 1.5340
Epoch 5/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 141s 402ms/step - accuracy: 0.3684 - loss: 1.4948 - val_accuracy: 0.4407 - val_loss: 1.3766
Epoch 6/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 143s 405ms/step - accuracy: 0.5494 - loss: 1.1586 - val_accuracy: 0.6441 - val_loss: 0.8544
Epoch 7/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 141s 400ms/step - accuracy: 0.7238 - loss: 0.7193 - val_accuracy: 0.8091 - val_loss: 0.6529
Epoch 8/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 118s 420ms/step - accuracy: 0.8578 -